In [15]:
%%capture
!pip install fastapi nest-asyncio pyngrok uvicorn python-multipart openai sentence_transformers

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


!./ngrok authtoken REDACTED
!apt-get install screen
!screen -dm bash -c 'ngrok http -region=us -hostname=REDACTEDD 8000'


In [ ]:
import os
import openai
import requests

openai.api_key = "REDACTED"


In [ ]:
import pickle
import pandas as pd
import numpy as np
def storeData(db):
    # Its important to use binary mode
    dbfile = open('tempPickle', 'ab')
      
    # source, destination
    pickle.dump(db, dbfile)                     
    dbfile.close()
    
    
def loadData():
    # for reading also binary mode is important
    dbfile = open('allData', 'rb')     
    db = pickle.load(dbfile)
    print(len(db))
    dbfile.close()
    return db

df=loadData()

for k,v in df['Calories (KCal)'].items():
  try:
    if v>100000:
      df['Calories (KCal)'][k]=v/1000
  except:
    pass

newCal=[]
import re
for k,v in df['Servings'].items():
  try:
    serv=int(re.match("[\d*]*(\d+)",str(v))[0])
  except:
    serv=1
  newCal.append(df.loc[k]['Calories (KCal)']/serv)
df['Real Cal']=pd.Series(newCal)
  

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
df['babbage_search'] = model.encode(np.array(df['Recipe Title'])).tolist()
npEmbeds=np.array([np.array(x) for x in df['babbage_search']]).astype(np.float32)


In [ ]:
import time
def get_embedding(text, engine="text-similarity-davinci-001"):
   time.sleep(0.03)
   return openai.Embedding.create(input = [text], engine=engine)['data'][0]['embedding']
 
#df['babbage_similarity'] = df[0].apply(lambda x: get_embedding(x, engine='text-similarity-babbage-001'))
df['babbage_search'] = df['Recipe Title'].apply(lambda x: get_embedding(x, engine='text-search-babbage-001'))


#df['babbage_similarity'] = df.babbage_similarity.apply(eval).apply(np.array)
#df['babbage_search'] = df.babbage_search.apply(eval).apply(np.array)

In [ ]:
df.babbage_similarity

0      [0.0031960606575012207, -0.0038988152518868446...
1      [0.015359480865299702, 0.02033042721450329, -0...
2      [-0.018149876967072487, 0.03496921807527542, -...
3      [-0.003398115048184991, -0.0031962876673787832...
4      [-0.000307851645629853, 0.02552475966513157, -...
                             ...                        
987    [0.009508182294666767, 0.005578523967415094, -...
988    [-0.01745978556573391, 0.017623573541641235, -...
989    [0.00212485040538013, 0.0034376562107354403, 0...
990    [0.002542107133194804, -0.0027599416207522154,...
991    [-0.0139173474162817, -0.0037985036615282297, ...
Name: babbage_similarity, Length: 992, dtype: object

In [ ]:
from openai.embeddings_utils import get_embedding, cosine_similarity
 
def search_reviews(df, product_description, n=3, pprint=True):
   embedding = get_embedding(product_description, engine='text-search-davinci-query-001')
   df['similarities'] = df['babbage_search'].apply(lambda x: cosine_similarity(x, embedding))
   res = df.sort_values('similarities', ascending=False).head(n)
   return res
 
res = search_reviews(df, 'fruit', n=10)


In [8]:
import requests
from bs4 import BeautifulSoup
import re


headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Microsoft Edge";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'Upgrade-Insecure-Requests': '1',
    'Origin': 'https://cosylab.iiitd.edu.in',
    'Content-Type': 'application/x-www-form-urlencoded',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36 Edg/99.0.1150.39',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://cosylab.iiitd.edu.in/recipedb/',
    'Accept-Language': 'en-US,en;q=0.9',
}

data = {
    'page': str(1),
    'autocomplete_region': '',
    'autocomplete_cuisine': '',
    'autocomplete_recipe': ''
}

def filterInstruct(url):
  global headers
  global data

  text=requests.post(url, headers=headers, data=data).text
  instruct=re.findall("instruction.*$",text,re.MULTILINE)[0][len('instructionToPara = " | '):]
  instruct= instruct[:re.search(r'-{10}', instruct).start()-2]
  instruct= re.sub("\| ", "\n",instruct)
  instruct= re.sub("\t", "  ",instruct)
  return instruct

def getInstructions(query):
  global headers
  data = {
      'page': str(1),
      'autocomplete_region': '',
      'autocomplete_cuisine': '',
      'autocomplete_recipe': query
  }

  reqs=requests.post('https://cosylab.iiitd.edu.in/recipedb/search_recipe', headers=headers, data=data)
  soup = BeautifulSoup(reqs.text, 'html.parser')
  
  urls = []
  for link in soup.find_all('a'):
    try:
      if "search_recipeInfo" in link.get('href'):
        data = {
            'page': str(1),
            'autocomplete_region': '',
            'autocomplete_cuisine': '',
            'autocomplete_recipe': ''
        }
        return filterInstruct('https://cosylab.iiitd.edu.in'+link.get('href'))

        
        break
    except:
      pass
  return "None"


In [ ]:
from tensorflow.python.framework.tensor_conversion_registry import get
from fastapi.responses import FileResponse
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
from fastapi import FastAPI, File, UploadFile
from fastapi import Response
from pydantic import BaseModel
import asyncio
from time import sleep
import re
from typing import List
import numpy as np
from datetime import datetime, timedelta
import json

import urllib.request
from PIL import Image
from bs4 import BeautifulSoup
import requests
import cv2


import io
app = FastAPI()


app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

class SurveyIn(BaseModel):
        age: str
        gender:  str
        height:  str
        weight:  str
        questions: List[str] = []
    
class MealsOut(BaseModel):
      meals: List[str] = []

class MealDataIn(BaseModel):
      id: str

class RecipeOut(BaseModel):
      recipe: str
  
class MealDataOut(BaseModel):
      nutrition: List[str] = [] #["vitamin C^0.45g",...]

class EatenIn(BaseModel):
      eaten: List[str] = []

class StatsIn(BaseModel):
      id: str

class StatOutModel(BaseModel):
      stat: List[str] = []
  
class Consistency(BaseModel):
      consistency: List[str] = []

userDict={}

@app.get("/recipe", response_model=RecipeOut)
def recipe(id: int = 0):
    # media_type here sets the media type of the actual response sent to the client.
    global meals
    return {"recipe": getInstructions(meals[id]) }


@app.get("/consistency", response_model=Consistency)
def consistency(answers: EatenIn):
    
    answers=json.loads(answers.json())
    eaten=[(1 if x=="1" else 0) for x in answers['eaten']]

    value=len(eaten) - 1 - eaten[::-1].index(1)
    b=[]
    i=0
    while i<len(eaten):
      b.append(str((eaten[i]+eaten[i+1]+eaten[i+2])/3*100)+'%')
      i+=3
    return {"consistency": b[:(value/3)]}




@app.get("/statistics", response_model=StatOutModel)
def calories(id: int = 0):    
    return {"stat": getCalories()}

@app.get("/image", responses = {200: {"content": {"image/png": {}}}}, response_class=Response)
async def get_image(id: int = 0):
    # media_type here sets the media type of the actual response sent to the client.
    global meals
    if id<0:
      id=0
    query = meals[id]
    query=f'"{query}"'
    hashedName=abs(hash(query)) % (10 ** 8)
    try:
      img = Image.open(f'/content/{hashedName}.png', mode='r')
    except:
      html_page = requests.get(f'https://www.google.com/search?q={query}&hl=en&tbm=isch&sxsrf=APq-WBvwYw7CNUiZc6PrdsAMkiCgIOVE1A%3A1647140742473&source=hp&biw=1280&bih=648&ei=hl8tYsyIGvWoqtsPo4Oa2AI&iflsig=AHkkrS4AAAAAYi1tlmv_zjSF4doH-hSjrDhlVjtmsHbm&ved=0ahUKEwjM4-O2jcL2AhV1lGoFHaOBBisQ4dUDCAc&uact=5&oq={query}&gs_lcp=CgNpbWcQAzIHCCMQ7wMQJzIICAAQgAQQsQMyCAgAEIAEELEDMggIABCABBCxAzIICAAQgAQQsQMyCAgAEIAEELEDMggIABCABBCxAzIICAAQgAQQsQMyCAgAEIAEELEDMggIABCABBCxAzoFCAAQgAQ6CAgAELEDEIMBOgsIABCABBCxAxCDAVAAWO8OYIkSaABwAHgAgAFjiAGgB5IBAjEwmAEAoAEBqgELZ3dzLXdpei1pbWc&sclient=img')
      soup = BeautifulSoup(html_page.content, 'html.parser')
      url=soup.find_all('img')[2]['src']

      urllib.request.urlretrieve(url,f'/content/{hashedName}.png')
      img = Image.open(f'/content/{hashedName}.png', mode='r')

    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    #img = cv2.resize(img, (224,224))
    #img = img.reshape(224,224,3)
    # media_type here sets the media type of the actual response sent to the client.
    return Response(content=img_byte_arr, media_type="image/png")

def getMeal(questions):
  possibilities=[]
  for i in range(3):
    row=[]
    for j in range(3):
      possibilities.append(get_candidate_foods(questions,i,j))
    #possibilities.append(row)
  meals=[]
  for i in range(21):
    for j in range(9):
      meals.append(possibilities[j].sample()['Recipe Title'].tolist()[0])
  return meals

def parse_data(data):
  global meals
  age=int(data['age']) 
  gender=data['gender']#1 m 2 f 3 nb
  height=data['height'].split("'") # 5'8 
  height=(int(height[0])*12+int(height[1]))*0.0254 #
  weight=int(data['weight'])*0.453592 # 50      (lbs)
  questions=[int(x) for x in data['questions']]
  activityLevel=((questions.pop(3)-1)/10+1)
  #therapist=[int(x) for x in data['therapist']]

  std,schedule=make_calorie_schedule(age, gender, height, weight, activityLevel)
  questions=questions[:3]
  questions.append(1 if std<0 else 0)
  return questions
  

@app.post('/getMeals', response_model=MealsOut)
async def getMeals(answers: SurveyIn):
  global meals
  global userDict
  answers=json.loads(answers.json())
  userDict=answers
  questions=parse_data(answers)

  meals = getMeal(questions)

  return {
        "meals": meals
      }

@app.api_route("/", methods=["GET", "POST"])
async def root():
    return 'Server Operational'

import nest_asyncio
from pyngrok import ngrok
import uvicorn
nest_asyncio.apply()
uvicorn.run(app, port=8000)

In [ ]:
class Action(object):
    def __init__(self, move):
        self.move = move
    
    def __eq__(self, other):
        return self.move == other.move
        
    def __hash__(self):
        return abs(hash(self.move)) % (10 ** 8)

class MealPlan(object):
    def __init__(self, pos, target, currCal = 0, depth=0):
        self.target = target
        self.pos = pos
        self.depth = depth
        self.currCal = currCal
        self.actions = [Action(x) for x in possibilities[depth].index.tolist()]
    
    def perform(self, action):
        global df
        self.pos.append(action.move)
        self.currCal+=df.loc[action.move]['Real Cal']
        return MealPlan(self.pos, self.target, self.currCal, self.depth)
        
    def reward(self, parent, action):
        if self.depth==10:
            return 1000000 - (self.target-self.currCal)**2
        else:
            return 10000-(self.target*(self.depth+1)/10-self.currCal)**2
            
    def is_terminal(self):
        return self.depth==8
            
    def __eq__(self, other):
        return all(self.pos == other.pos)
        
    def __hash__(self):
        return abs(hash(str(self.pos))) % (10 ** 8)

In [ ]:
%cd /content/mcts/mcts
from tree_policies import *
from utils import *
from backups import *
from default_policies import *
from graph import *
from mcts import *

mcts = MCTS(tree_policy=UCB1(c=1.41), 
            default_policy=RandomKStepRollOut(3),
            backup=monte_carlo)

In [ ]:
def getMeals():
  stats=[]
  for i in makeCalorieSchedule(15,"male",1.69,52,1.55):
    myState=[]
    currCal=0
    depth=0
    for i in range(9):
      root = StateNode(parent=None, state=MealPlan(myState.copy(),1900,currCal,depth))
      best_action=mcts(root)
      myState.append(best_action.move)
      currCal+=df.loc[best_action.move]['Real Cal']
      print(currCal)
      depth+=1
    stats.append[myState]
  return df.loc[stats]

In [65]:

import tensorflow as tf

def getSimScore(term):
  global npEmbeds
  a=npEmbeds
  b=model.encode(term).astype(np.float32)

  a=tf.constant(a)
  b=tf.constant(b)

  similarity = tf.reduce_sum(a[:, tf.newaxis] * b, axis=-1)
  similarity /= tf.norm(a[:, tf.newaxis], axis=-1) * tf.norm(b, axis=-1)

  return similarity.numpy()[:,0]


def sumSims(terms,weights):
  sum=np.ones(118160)
  count=0
  for qw in zip(terms, weights):
    if qw[1]==0:
      continue
    
    sum=np.multiply(sum, getSimScore(qw[0])*qw[1])
    if count>=7:
      sum=np.sum([sum,getSimScore(qw[0])*qw[1]*3],axis=0)
    count+=1
  return sum



import re
def get_candidate_foods(
    questionss,
    meal,#0=bkfst;1=lunch;2=dinner
    entree, #0=entree, 1=side , 2=fruit
    n=15,
    ):
    global df
    global meals
    weights = [[0, 0, 0, 0, 0],
               [0, 0],
               [0, 0, 0, 0, 0],
               [0, 0],
               [1, 0, 0],
               [0, 1, 0],
               [0, 0, 1],
               [1, 0, 0],
               [0, 1, 0],
               [0, 0, 1]]
    terms=["food that is spicy",
           "contains meat",
           "sweet food",
           "high calorie food",
           "breakfast food",
           "lunch food",
           "dinner food",
           "entree",
           "appetizer",
           "fruit"
           ]
    questions=questionss.copy()
    newWeights=[]
    questions[0]-=1
    questions[2]-=1
    for i in range(4):
      newWeights.append(weights[i][questions[i]])
    for i in range(3):
      newWeights.append(weights[i+4][meal])
    for i in range(3):
      newWeights.append(weights[i+7][entree])
    #print(newWeights)
    df['similarities'] = list(sumSims(terms, newWeights))
    res = df.sort_values('similarities', ascending=False).head(n)
    excluded = ['food']
    sstring='|'.join(excluded)
    res = res[[not x for x in res['Recipe Title'].str.contains(sstring, flags=re.IGNORECASE,na=False)]]
    return res



import numpy
from matplotlib import pyplot as plt

def make_calorie_schedule(age, gender, height, weight, activityMultiplier):

  def bmi(weight, height):
    effectiveBMI = weight/(height)**2
    return effectiveBMI

  #height is in m, weight is in kg
  # print(bmi(52.62, 1.69))

  #gender = "male"
  if gender=='1':
    gender='male'
  elif gender=='2':
    gender='female'
  

  if gender == "female":
    ageList = [3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18, 18.5, 19, 19.5, 20, 20.5, 21, 21.5, 22, 22.5, 23]
    bmiList = [16, 16.1, 16.2, 16, 15.9, 16.1, 16.7, 17.1, 17.2, 17.3, 17.5, 18.4, 18.7, 19.2, 18.9, 19.5, 20.5, 20.4, 20.8, 21.2, 21.7, 21.6, 21.9, 22.4, 21.9, 22.5, 21.8, 22.1, 21.8, 22.1, 21.8, 21.4, 22.0, 22.1, 21.1, 22.0, 21.4, 21.5, 21.2, 22.0, 21.4]
    stdDevList = [1.28, 1.92, 2.01, 1.84, 2.06, 2.28, 2.61, 2.88, 3.13, 3.13, 3.48, 3.67, 3.49, 3.53, 4.13, 4.03, 4.56, 4.52, 3.90, 5.29, 4.34, 4.37, 4.30, 4.15, 4.45, 3.45, 3.41, 3.48, 3.30, 3.46, 3.17, 3.33, 3.13, 3.62, 3.21, 3.04, 2.71, 2.84, 2.81, 4.04, 2.76]
  else:#if gender == "male":
    ageList = [3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18, 18.5, 19, 19.5, 20, 20.5, 21, 21.5, 22, 22.5, 23]
    bmiList = [16.4, 16.1, 16.2, 16.1, 16.2, 16.5, 16.6, 17.0, 17.0, 17.3, 17.3, 18.6, 18.6, 18.9, 19.1, 19.6, 20.6, 19.8, 20.5, 21.2, 20.7, 21.1, 21.6, 21.5, 23.0, 22.4, 22.7, 22.6, 22.8, 23.1, 23.0, 23.5, 24.1, 23.6, 23.7, 24.0, 23.5, 24.1, 23.5, 24.1, 23.5]
    stdDevList = [3.00, 1.69, 1.90, 2.07, 1.83, 2.51, 2.50, 2.67, 3.12, 3.16, 2.77, 3.95, 3.85, 3.60, 3.80, 4.69, 4.62, 4.04, 4.11, 4.84, 4.10, 3.86, 4.06, 4.03, 4.27, 4.06, 4.05, 4.04, 3.72, 3.74, 4.37, 4.74, 4.69, 3.46, 3.38, 3.38, 2.99, 3.23, 2.60, 3.39, 3.19]
  #age is subject to change - REMINDER TO IMPLEMENT ACTIVITY MULTIPLIER IN APP
  #activityMultiplier = 1.55
  #age = 15
  #height = 1.69
  #weight = 80

  age=int(np.clip(age,3,23))
  ageIndex = ageList.index(age)
  #print(ageList[ageIndex], bmiList[ageIndex], stdDevList[ageIndex])

  #calculate number of standard deviations

  numStdDev = ((bmi(weight, height))-bmiList[ageIndex])/stdDevList[ageIndex]

  #parameters for the calorie intake/18 days function
  bodyFatPercentage = (bmi(weight, height) * 1.2) + (0.23 * age) - 16.2

  desiredWeight = (bmiList[ageIndex])*height**2
  weightPerDay = (desiredWeight-weight)/21

  #calculate lean factor
  if bodyFatPercentage <= 14:
    leanFactor = 1
  elif 20 >= bodyFatPercentage > 14:
    leanFactor = 0.95
  elif 28 >= bodyFatPercentage > 20:
    leanFactor = 0.9
  elif 28 > bodyFatPercentage:
    leanFactor = 0.85

  weightList = []
  dateTime = []
  for i in range(21):
    weightList.append(weight + i*weightPerDay)
    dateTime.append(i+1)

  calorieList = []
  for i in range(21):
    if gender == "male":
      bmr = weightList[i] * 24 * leanFactor
    else: #elif gender == "female":
      bmr = weightList[i] * 24 * 0.9 * leanFactor
    calorieList.append(bmr * activityMultiplier)

  return numStdDev, calorieList
  # print(weightList)
  # print(calorieList)
  # print(dateTime)

  #print(len(ageList), len(bmiList))
  # plt.plot(dateTime, calorieList)